In [15]:
import datetime, os, pathlib, platform, sys
import bitstring
import pandas as pd
import numpy as np

from PIL import (
    Image
)

from typing import (
    Callable
) 

from utils import (
    FILE,
    DATASET,
    STATS,
    examine_dataset,
    get_file_path
)


seed: int = 14

# set up pandas display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# print library and python versions for reproducibility
print(
    f'''
    Last Execution: {datetime.datetime.now()}
    python:\t{platform.python_version()}

    \tnumpy:\t\t{np.__version__}
    \tpandas:\t\t{pd.__version__}
    '''
)


    Last Execution: 2022-11-09 22:25:58.794386
    python:	3.10.4

    	numpy:		1.23.1
    	pandas:		1.4.4
    


In [2]:
data_path_1: str = '../../data/csv/'
data_sets_1: list = [
    'Darknet_experiments_base.csv'
]

file_path_1: callable = get_file_path(data_path_1)
file_set_1: list = list(map(file_path_1, data_sets_1))
current_job: int = 0

dataset_1 = examine_dataset(0, file_set_1, data_sets_1)

Dataset 0/1: We now look at ../../data/csv/Darknet_experiments_base.csv


Loading Dataset: ../../data/csv/Darknet_experiments_base.csv
	To Dataset Cache: ./cache/Darknet_experiments_base.csv.pickle


        File:				../../data/csv/Darknet_experiments_base.csv  
        Job Number:			0
        Shape:				(117620, 64)
        Samples:			117620 
        Features:			64
    


In [3]:
dataset_1[DATASET].head()

,Protocol,Flow Duration,Total Fwd Packet,Total Bwd packets,Total Length of Fwd Packet,Total Length of Bwd Packet,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Packet Length Min,Packet Length Max,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,Down/Up Ratio,Average Packet Size,Fwd Segment Size Avg,Bwd Segment Size Avg,Bwd Packet/Bulk Avg,Bwd Bulk Rate Avg,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Bytes,FWD Init Win Bytes,Bwd Init Win Bytes,Fwd Act Data Pkts,Fwd Seg Size Min,Idle Mean,Idle Std,Idle Max,Idle Min,Traffic Type,Application Type
0,6,119384497,767,2027,90681,2448831,543,0,118.228162,224.244409,1420,0,1208.106068,401.596755,21271.706660,23.403374,42744.18081,583316.42250,10430835,0,119384314,155854.1958,1.106494e+06,10431320,39,119384354,58926.13722,685064.8872,10431069,0,1,15340,40540,6.424620,16.978754,0,1420,908.785331,606.092512,367348.1336,0,0,0,1032,2794,2,909.110594,118.228162,1208.106068,336,150679,0,32,876,65535,65535,166,20,1.456330e+15,3.683488e+07,1.456330e+15,1.456330e+15,Tor,audio-streaming
1,6,119754701,1021,2537,171290,2922270,1460,0,167.766895,280.092409,1420,0,1151.860465,464.838481,25832.472330,29.710733,33667.33230,443717.51820,10951164,0,119754701,117406.5696,8.231362e+05,10951327,22,119754258,47221.71057,525618.1358,10951341,3,1,20420,50740,8.525761,21.184972,0,1460,869.374262,612.115804,374685.7581,0,0,0,1271,3558,2,869.618606,167.766895,1151.860465,421,94293,0,48,821,65535,65535,292,20,1.456330e+15,3.558961e+07,1.456330e+15,1.456330e+15,Tor,audio-streaming
2,6,118908117,683,1662,77649,2001627,543,0,113.688141,221.086482,1420,0,1204.348375,411.993028,17486.409280,19.721110,50728.71886,635683.39510,11346294,0,118908117,174352.0777,1.170126e+06,11346697,20,118907832,71588.09874,755418.0467,11346612,0,1,13660,33240,5.743931,13.977179,0,1420,886.538363,616.479582,380047.0751,0,0,0,800,2345,2,886.916418,113.688141,1204.348375,275,168275,0,33,853,65535,65535,142,20,1.456330e+15,3.301057e+07,1.456330e+15,1.456330e+15,Tor,audio-streaming
3,6,739728,2,2,543,543,543,0,271.500000,383.958982,543,0,271.500000,383.958982,1468.107196,5.407393,246576.00000,426907.22530,739526,55,739728,739728.0000,0.000000e+00,739728,739728,739526,739526.00000,0.0000,739526,739526,1,40,40,2.703696,2.703696,0,543,325.800000,297.413349,88454.7000,0,0,0,2,4,1,407.250000,271.500000,271.500000,0,0,0,135,135,41180,65535,0,20,1.456330e+15,5.227810e+05,1.456330e+15,1.456330e+15,Tor,audio-streaming
4,6,149270,2,2,565,565,565,0,282.500000,399.515331,565,0,282.500000,399.515331,7570.174851,26.797079,49756.66667,85987.11268,149046,43,149270,149270.0000,0.000000e+00,149270,149270,149046,149046.00000,0.0000,149046,149046,1,40,40,13.398540,13.398540,0,565,339.000000,309.463245,95767.5000,0,0,0,2,4,1,423.750000,282.500000,282.500000,0,0,0,141,141,41180,65535,0,20,1.456330e+15,1.052330e+05,1.456330e+15,1.456330e+15,Tor,audio-streaming


In [4]:
APPLICATION_TYPE: str = 'Application Type'
TRAFFIC_TYPE: str = 'Traffic Type'

In [5]:
# get classes in Traffic Type column
dataset_1[DATASET][TRAFFIC_TYPE].unique()

array(['Tor', 'VPN', 'Regular'], dtype=object)

In [6]:
dataset_1[DATASET][TRAFFIC_TYPE].value_counts()

Regular    93309
VPN        22919
Tor         1392
Name: Traffic Type, dtype: int64

In [7]:
dataset_1[DATASET][APPLICATION_TYPE].unique()

array(['audio-streaming', 'browsing', 'chat', 'file-transfer', 'email',
       'p2p', 'video-streaming', 'voip'], dtype=object)

In [8]:
dataset_1[DATASET][APPLICATION_TYPE].value_counts()

p2p                48520
browsing           32808
audio-streaming    14754
file-transfer       9341
chat                4951
video-streaming     4709
voip                1465
email               1072
Name: Application Type, dtype: int64

In [9]:
df = dataset_1[DATASET]

In [10]:
vals = df.values

In [11]:
vals.shape

(117620, 64)

In [12]:

def convert_samples_to_binary(df: pd.DataFrame, precision: 32 or 64 = 64) -> np.ndarray:
    '''
        Take every sample in the df and convert it into a 2d array with: 
            each sample being a row,
            each column contains the elements of that sample's binary representation with the given precision
    
        binary representation is 1 bit for sign, 8 bits for exponent, precision - 9 bits for mantissa

        This will return a 3d array with the following shape:
            (number of samples, number of features, precision)


    '''

    out = np.zeros((df.shape[0], df.shape[1], precision))

    # get the values of the dataframe
    vals = df.values

    # convert the values to binary
    for i, sample in enumerate(vals):
        for j, feature in enumerate(sample):
            for k, bit in enumerate(bitstring.BitArray(float=feature, length=precision).bin):
                out[i][j][k] = int(bit)

    return out


In [19]:
def convert_samples_to_binary(df: pd.DataFrame, precision: 32 or 64 = 64, one: int = 128, zero: int = 0) -> np.ndarray:
    '''
        Take every sample in the df and convert it into a 2d array with: 
            each sample being a row,
            each column contains the elements of that sample's binary representation with the given precision
    
        binary representation is 1 bit for sign, 8 bits for exponent, precision - 9 bits for mantissa

        This will return a 3d array with the following shape:
            (number of samples, number of features, precision)


    '''

    out = []

    # get the values of the dataframe
    vals = df.values

    # convert the values to binary
    for i, sample in enumerate(vals):
        sample_out = []
        for j, feature in enumerate(sample):
            feature_out = [one if b == '1' else zero for b in bitstring.BitArray(float=feature, length=precision).bin]
            sample_out.append(feature_out)
        out.append(sample_out)

    return np.array(out, dtype=np.uint8)


def save_sample_as_image(sample: np.ndarray, path: str, name: str) -> None:
    '''
        Save the sample as an image
    '''

    # create the path if it doesn't exist
    pathlib.Path(path).mkdir(parents=True, exist_ok=True)

    # create the image
    img = Image.fromarray(sample, 'L')

    # save the image
    img.save(f'{path}{name}.png')

    return None


def generate_binary_images_from_df(
    df: pd.DataFrame, 
    label: str, 
    precision: 32 or 64 = 64, 
    path: str = './', 
    name: str = 'binary_image', 
    one: int = 128, 
    zero: int = 0
) -> None:
    '''
        Generate binary images from a dataframe

        The dataframe must only have 1 label column with all other labels removed
        The data must be in either integer or float format
    '''

    # get the labels from the samples
    labels = df[label].values

    # remove the label column from the dataframe
    new_df = df.copy().drop(columns=[label])


    # convert the samples to binary
    samples = convert_samples_to_binary(new_df, precision, one = one, zero = zero)

    # save the samples as images
    for i, sample in enumerate(samples):
        sample_name = f'{name}_{labels[i]}_{i}'

        save_sample_as_image(sample, path, sample_name)


    return None

In [20]:
generate_binary_images_from_df(
    df.copy().drop(columns=[APPLICATION_TYPE]).head(10),
    TRAFFIC_TYPE,
    precision = 64
)

In [41]:
df2 = df.copy()
df2 = df2.drop(columns=[TRAFFIC_TYPE, APPLICATION_TYPE])

In [42]:
binary_samples = convert_samples_to_binary(df2, precision=32)

In [43]:
def convert_samples_to_binary(df: pd.DataFrame, precision: 32 or 64 = 64) -> np.ndarray:
    '''
        Take every sample in the df and convert it into a 2d array with: 
            each sample being a row,
            each column contains the elements of that sample's binary representation with the given precision
    
        binary representation is 1 bit for sign, 8 bits for exponent, precision - 9 bits for mantissa

        This will return a 3d array with the following shape:
            (number of samples, number of features, precision)


    '''

    out = []

    # get the values of the dataframe
    vals = df.values

    # convert the values to binary
    for i, sample in enumerate(vals):
        sample_out = []
        for j, feature in enumerate(sample):
            feature_out = [int(b) for b in bitstring.BitArray(float=feature, length=precision).bin]
            sample_out.append(feature_out)
        out.append(sample_out)

    return np.array(out)


In [44]:
binary_samples = convert_samples_to_binary2(df2, precision=32)